In [ ]:
import sympy as sp
sp.init_printing()
import numpy as np
import matplotlib.pyplot as plt
import setup_plots
import matplotlib.patheffects as path_effects
import torchvision.models as models
import torch


In [ ]:
setup_plots.setup()

In [ ]:
plt.rcParams['figure.dpi'] = 200

In [ ]:
colors=plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
def get_mobilenet_target_mats():
    target_mats = []
    # Load the model
    model = models.mobilenet_v2(pretrained=True)
    # Put moel into eval mode
    model.eval()
    for layer in model.classifier:
        if isinstance(layer, torch.nn.Linear):
            # Obtain the weights of this layer
            weights = layer.weight.detach().numpy()
            target_mats.append(weights)
    return target_mats
mats = get_mobilenet_target_mats()[0]

In [ ]:
def calc_sigmas_Hankel(A,K):
    
    d_in = A.shape[1]
    boundaries = d_in/K*np.arange(1,K)
    boundaries_in = np.round(boundaries).astype(int)
    #print("d_in",d_in)
    #print("bounds_in",boundaries_in)

    d_out = A.shape[0]
    boundaries = d_out/K*np.arange(1,K)
    boundaries_out = np.round(boundaries).astype(int)
    #print("d_out",d_out)
    #print("bounds_out",boundaries_out)
    
    sigmas = []
    for i in range(K-1):
        sigmas.append(np.linalg.svd(A[boundaries_out[i]:,:boundaries_in[i]],compute_uv=False))
    return sigmas
    
K = 20
sigmas = calc_sigmas_Hankel(mats,K)

In [ ]:
w = setup_plots.textwidth
#plt.figure(figsize = [w,w*3/4])
fig, ax = plt.subplots(figsize = [w,w*2/4])

max_sigmas = max([np.max(sig) for sig in sigmas])

[P,M] = mats.shape

alphas = [0.4,0.5,0.6]
gammas = [0.13,0.05,0.02]

plt.plot([],[],label=r"Approximation")
ax.set_prop_cycle(None) #reset cycler
x = np.linspace(0,20)
for i,gamma in enumerate(gammas):
    d = gamma*(K-x)*(x)*P*M/((K**2)*min(P,M))
    plt.plot(x,d,color=colors[i],alpha=0.5,
            label=r"$\gamma = {g:.2F}$".format(g=gamma))

plt.plot([],[],label=r"Actual $d$")
x = np.arange(21)
for i, alpha in enumerate(alphas):
    eps = alpha*max_sigmas
    d =[np.count_nonzero(sig>eps) for sig in sigmas]
    d.append(0)
    d.insert(0,0)
    plt.scatter(x,np.array(d),marker=str(i+1),color=colors[i],
             label=r"$\alpha = {a:.2F}$".format(a=alpha))
    #marker=(5, 2, 0) #



plt.grid()
plt.xlabel(r"$k$")
plt.ylabel(r"$d_k$")

legend = fig.legend(loc='center right',ncol=2,columnspacing=0.3,handletextpad=0.4)#handlelength=1.5
#legend = plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5),ncol=2,columnspacing=1)


legend.get_children()[0].get_children()[1].get_children()[0].get_children()[0].get_children()[0].width=-6
legend.get_children()[0].get_children()[1].get_children()[0].get_children()[0].get_children()[0].get_children()[1].set(visible=False)
legend.get_children()[0].get_children()[1].get_children()[0].get_children()[0].get_children()[0].get_children()[0].set(visible=False)
legend.get_children()[0].get_children()[1].get_children()[1].get_children()[0].get_children()[0].width=5
legend.get_children()[0].get_children()[1].get_children()[1].get_children()[0].get_children()[0].get_children()[1].set(visible=False)
legend.get_children()[0].get_children()[1].get_children()[1].get_children()[0].get_children()[0].get_children()[0].set(visible=False)

#legend.get_children()[0].get_children()[1].get_children()[0].get_children()[0].get_children()[1].set(text="Approx")
plt.subplots_adjust(right=0.63)
plt.savefig("approx_degree.pdf",bbox="tight",bbox_inches='tight')

# Some helping function

Transform the sum into a polynomial

calcaultes $$\sum^N_{n=1} arg(n) $$  for $$arg = \sum_{p=0}^5 a_p n^p$$

In [ ]:
def transfrom_sum(arg,n,N,n_start=None):
    #Take a sum consisting of polynomilas in n and calcualtes the overall polynomial using the Faulhabersche_Formel
    #Knuth https://arxiv.org/abs/math/9207222
    #calcaultes $$\sum^N_{n=1} exp$$  for $$exp = \sum_{p=0}^5 a_p n^p$$
    #Parameters:
    #    arg:   expression in the sum
    #    n:     running variable
    #    N:     upper end of sum
    #returns:
    # expression 
    arg = arg.expand().collect(n)
    expression = 0
    for i in range(6):
        #faulhaber coeffs form: https://de.wikipedia.org/wiki/Faulhabersche_Formel 
        #look below for more details
        coeffs = {0:N,
              1:1/sp.S(2)*N**2+1/sp.S(2)*N,
              2:1/sp.S(3)*N**3+1/sp.S(2)*N**2+1/sp.S(6)*N,
              3:1/sp.S(4)*N**4+1/sp.S(2)*N**3+1/sp.S(4)*N**2,
              4:1/sp.S(5)*N**5+1/sp.S(2)*N**4+1/sp.S(3)*N**3-1/sp.S(30)*N,
              5:1/sp.S(6)*N**6+1/sp.S(2)*N**5+5/sp.S(12)*N**4-1/sp.S(12)*N**2}


        expression = expression + (arg.coeff(n,i))*coeffs[i] 
        if n_start:
            expression = expression - (arg.coeff(n,i))*coeffs[i].subs({N:n_start-1})
            #display((arg.coeff(n,i))*coeffs[i].subs({N:n_start-1}))
    #test for higher orders
    for i in range(6,100):
        if arg.coeff(n,i) !=0:
            raise NotImplementedError("n^"+str(i)+" is not implemented")
    return expression

In [ ]:
K,m,p,M,P = sp.symbols('K m p M P',positive=True)
N = sp.symbols('N',positive=True) #for square matrix
dk,dk1 = sp.symbols('d_k d_{k+1}',positive=True)
gamma = sp.symbols('gamma',positive=True)
k = sp.symbols('k')

cost =2*dk1*dk+2*dk1*M/K+2*P/K*dk+P*M/K**2

In [ ]:
d_prod = gamma*(K-k+1)*(k-1)*P*M/(K**2*N).subs(N,M) #here N is min(M,P)
d_prod_k = d_prod
d_prod_k1 = d_prod.subs(k,k+1)

arg_prod = cost.subs({dk:d_prod_k,dk1:d_prod_k1})
arg_prod

In [ ]:
cost_total_prod = transfrom_sum(arg_prod,k,K)

In [ ]:
cost_of_K = cost_total_prod.expand().collect(K)
cost_of_K

In [ ]:
sp.pycode(cost_of_K)

In [ ]:
cost_of_K.subs(K,1)

Now set for square matrix

In [ ]:
cost_total_prod=cost_total_prod.subs({M:N,P:N}).expand().collect(K)
cost_total_prod

In [ ]:
w = 0.75*setup_plots.textwidth
fig, ax = plt.subplots(figsize = [w,w*3/4])
col = '#1f77b4' #'#ff7f0e'
ax.set_xscale('log',basex=2)
ax.set_yscale('log',basey=10)
xmin = 0.8
xmax = 2**10
ymin = 10**3.8
ymax = 10**8

map_x = lambda x : np.log2(x/xmin)/np.log2(xmax/xmin)
map_y = lambda y: np.log10(y/ymin)/np.log10(ymax/ymin)

pos_x = np.array([2**7.8,2**8.2,2**7.8,2**7,2**6,2**5])


def annotoate(xs,ys,text,x,dy_dx=1):
    i = np.count_nonzero(xs<x)#index of the first point to the right
    y = (x-xs[i])/(xs[i-1]-xs[i])*ys[i-1]+(x-xs[i-1])/(xs[i]-xs[i-1])*ys[i] 
    text = ax.text(
    x,
    y,
    text,
    rotation=np.arctan((ys[i]-ys[i-1])/(xs[i]-xs[i-1])*dy_dx)*180/np.pi,
    size="small",
    bbox=dict(facecolor="white", edgecolor="None", alpha=0.85,boxstyle="square",pad=0.1,mutation_aspect=-2),
    color=col,
    ha="center",
    va="center",
    transform=ax.transAxes
    )

f = sp.lambdify([K,(N,gamma)],cost_total_prod)
Ks = 2**np.arange(11)
Ks_sub = 2**np.linspace(0,11,40)
Nn = Ks[-1]
for n,gamma_tilde in enumerate(np.round(np.logspace(-2,0,6),2)):
    costs= f(Ks,(1024,gamma_tilde))
    plt.plot(Ks,costs,label=r"$\gamma = {g:.2F}$".format(g=gamma_tilde),color=col,linestyle='-')
    i = np.argmin(costs)
    plt.plot(Ks[i],costs[i],marker='o',markerfacecolor = "white",color=col)
    annotoate(map_x(Ks_sub),map_y(f(Ks_sub,(1024,gamma_tilde))),r"$\gamma = "+str(gamma_tilde)+r"$".format(g=gamma_tilde),map_x(pos_x[n]),dy_dx=1/1.5)

    
plt.grid()
plt.hlines(Nn**2,0,2**10,linewidth=1)
locs, labels = plt.xticks()
plt.xlim([xmin,xmax])
plt.ylim([ymin,ymax])

plt.xlabel("$K$")
plt.ylabel("Approximated Cost")

plt.savefig("cost_parameters.pdf",bbox="tight")